### Basic Agent
- example of basic chat agent using a custom tool
- custom tool scores pre-trained classifier on previously unseen data

In [1]:

from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import make_scorer

In [56]:
# chat agent

from config import api_key

apikey=api_key

import os

os.environ['OPENAI_API_KEY'] = apikey

In [73]:
from langchain.agents import Tool

def llm_model_score(string):
    
    test_data_path,model_path=string.split(",")
    
    import pickle
    
    # load test data
    
    test_data = pickle.load(open(test_data_path, 'rb'))
    
    X_test=test_data[[x for x in test_data.columns if x!='priceRange']]
    
    y_test=test_data['priceRange']
    
    # load model

    loaded_model = pickle.load(open(model_path, 'rb'))
    
    result = loaded_model.score(X_test, y_test)
    
    return str(result)


score_tool = Tool(
    name='llm_model_score',
    func= llm_model_score,
    description="Useful for when you need to score a trained model on test data. The input to this tool should be a comma separated list of length 2 of strings representing the path to the saved test data and model.  For example 'saved_data.sav','saved_model.sav'. "
)






In [38]:
# test function
print(llm_model_score(test_data_path='test_data.sav',model_path='best_model.sav'))

0.608


In [61]:
from langchain import OpenAI 
from langchain.chat_models import ChatOpenAI

# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)



In [74]:
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


tools = [score_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
#     llm=local_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory,
    handle_parsing_errors=True
)

In [70]:
question="""What is the score of the pretrained model saved at path best_model.sav on the new test data saved at path test_data.sav?"""

In [75]:
manual_react = f"""Question: What is the score of the pretrained model on the new test data?
Thought: I need to score the pretrained model that is saved at model path best_model1.sav on the new test data saved at the path test_data1.sav.
Action: score_tool['test_data1.sav','best_model1.sav']
Observation: 0.75.
Thought: The score returned was 0.75 so the model score on the test data is 0.75.
Action: Finish[model score 0.75]

Question:{question}"""

In [76]:
conversational_agent(manual_react)



> Entering new AgentExecutor chain...
{
    "action": "llm_model_score",
    "action_input": "test_data.sav,best_model.sav"
}
Observation: 0.608
Thought:{
    "action": "Final Answer",
    "action_input": "The score of the pretrained model saved at path best_model.sav on the new test data saved at path test_data.sav is 0.608."
}

> Finished chain.


{'input': "Question: What is the score of the pretrained model on the new test data?\nThought: I need to score the pretrained model that is saved at model path best_model1.sav on the new test data saved at the path test_data1.sav.\nAction: score_tool['test_data1.sav','best_model1.sav']\nObservation: 0.75.\nThought: The score returned was 0.75 so the model score on the test data is 0.75.\nAction: Finish[model score 0.75]\n\nQuestion:What is the score of the pretrained model saved at path best_model.sav on the new test data saved at path test_data.sav?",
 'chat_history': [],
 'output': 'The score of the pretrained model saved at path best_model.sav on the new test data saved at path test_data.sav is 0.608.'}